## import libraries

In [1]:
from compas.geometry import Point, Sphere
from compas_vol.microstructures import Voronoi
from compas_vol.primitives import VolSphere
from compas_vol.combinations import Intersection

In [2]:
import numpy as np
import meshplot as mp
from skimage.measure import marching_cubes
from compas_vol.utilities import add_cube

## random points cloud generation

In [3]:
nbr_pts = 16
coordinates = np.random.uniform(-30, 30, (nbr_pts,3))
pts = [Point(px,py,pz) for (px,py,pz) in coordinates]

## create volumetric object (CSG tree)

In [4]:
s = Sphere(Point(0,0,0), 25)
vs = VolSphere (s)
v = Voronoi(pts, thickness=2.0)
u = Intersection(vs, v)

## workspace (dense grid)

In [5]:
#workspace initialization
# lower and upper bounds
lbx, ubx = -30.0, 30.0
lby, uby = -30.0, 30.0
lbz, ubz = -30.0, 30.0
# resolution(s)
nx, ny, nz = 100, 100, 100
x, y, z = np.ogrid[lbx:ubx:nx*1j, lby:uby:ny*1j, lbz:ubz:nz*1j]
#voxel dimensions
gx = (ubx-lbx)/nx
gy = (uby-lby)/ny
gz = (ubz-lbz)/nz

## sample at discrete interval

In [6]:
dm = u.get_distance_numpy(x, y, z)

## generate isosurface(marching cube)

In [9]:
v, f, n, l = marching_cubes(dm, 0, spacing=(gx, gy, gz))
v += [lbx,lby,lbz]

## display mesh

In [10]:
print('use fc slider for cutting plane')
p = mp.plot(v, f, c=np.array([0,0.57,0.82]), shading={"flat":True, "roughness":0.4, "metalness":0.01, "reflectivity":1.0})
vs,ve = add_cube(lbx,ubx,lby,uby,lbz,ubz)
p.add_lines(np.array(vs), np.array(ve))

@mp.interact(fc=(0,120000))
def facefilter(fc=120000):
    p.update_object(faces=f[:fc])

use fc slider for cutting plane


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.300001…

interactive(children=(IntSlider(value=120000, description='fc', max=120000), Output()), _dom_classes=('widget-…